# NWB-Adapter (Simple)

This notebook demostrates how to use the new `dj-AttributeAdapter` feature to work with `NWB` objects.

In [1]:
# import datajoint, nwb modules
%matplotlib inline
import datajoint as dj
import os
import pynwb
from pynwb import NWBFile, NWBHDF5IO
from datetime import datetime
from dateutil.tz import tzlocal
import json
import numpy as np
import pathlib
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.environ['DJ_SUPPORT_ADAPTED_TYPES'] = 'TRUE'
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = 'TRUE'

# Objective

+ Build `dj.AttributeAdapter` for ***NWBFile*** object
+ Use this in a DataJoint table
+ Demo `.insert()` and `.fetch()`

## Step 1 - Create a DataJoint AttributeAdapter for NWB object

Basically we will need to define an object inhereted from `dj.AttributeAdapter` and instantiated with a variable name ***nwb_file***

In [3]:
exported_nwb_dir = dj.config['stores']['nwbstore']['location']

In [4]:
exported_nwb_dir

'./nwb_store/demo'

In [5]:
class NWBFileAdapter(dj.AttributeAdapter):
    attribute_type = 'filepath@nwbstore'  # nwbstore is some directory (either local or on cloud)
    
    def put(self, nwb):
        save_file_name = ''.join([nwb.identifier, '.nwb'])
        # save the file
        with NWBHDF5IO(os.path.join(exported_nwb_dir, save_file_name), mode='w') as io:
            io.write(nwb)
            print(f'Write NWB 2.0 file: {save_file_name}')
        # return the filepath to be inserted into DataJoint tables
        return os.path.join(exported_nwb_dir, save_file_name)
        
    def get(self, path):
        # read the nwb filepath and return an nwb file object back to the user
        return NWBHDF5IO(path, mode='r').read()

#### Instantiate for use as a datajoint type

In [6]:
nwb_file = NWBFileAdapter()

## Step 2 - Create a new schema ***export*** and NWB table

This ***NWB*** table specifies a primary key of `experiment.Session`, designed to store one NWB object (or NWBFile) per session

In [7]:
schema = dj.schema('demo_nwb_adapter')

Connecting root@127.0.0.1:3306


In [8]:
@schema
class NWB(dj.Manual):
    definition = """
    nwb_id: int
    ---
    nwb: <nwb_file> 
    """

In [9]:
NWB()

nwb_id,nwb


Note that the table definition above set the ***nwb*** attribute to be of type ***< nwb_file >***. 

Hence the reason for defining ***nwbfile*** as an instant of ***NWBAdapter*** - see Step 1

## Step 3 - Build an NWBFile

Here, we build a very simple NWB object using the `pynwb` package, for the sake of demonstration

In [10]:
# -- create an NWBFile (pynwb.file.NWBFile)
nwb = NWBFile(identifier='nwb_01',
              session_description='',
              session_start_time=datetime.strptime('2019-10-20', '%Y-%m-%d'),
              file_create_date=datetime.now(tzlocal()),
              experimenter='John Smith')

# -- add subject
nwb.subject = pynwb.file.Subject(subject_id='animal_01', sex='F')

In [11]:
nwb


root <class 'pynwb.file.NWBFile'>
Fields:
  experimenter: John Smith
  file_create_date: [datetime.datetime(2020, 5, 13, 16, 49, 3, 906224, tzinfo=tzlocal())]
  identifier: nwb_01
  session_start_time: 2019-10-20 00:00:00-05:00
  subject: subject <class 'pynwb.file.Subject'>
  timestamps_reference_time: 2019-10-20 00:00:00-05:00

## Step 4 -  Insert to the ***NWB*** table

In [12]:
NWB()

nwb_id,nwb


In [13]:
NWB.insert1({'nwb_id': 0, 'nwb': nwb})

Write NWB 2.0 file: nwb_01.nwb


In [14]:
NWB()

nwb_id,nwb
0,=BLOB=


### Now, fetch that NWB file back

In [15]:
fetched_nwb = (NWB & 'nwb_id=0').fetch1('nwb')

In [16]:
fetched_nwb


root <class 'pynwb.file.NWBFile'>
Fields:
  experimenter: ['John Smith']
  file_create_date: [datetime.datetime(2020, 5, 13, 16, 49, 3, 906224, tzinfo=tzoffset(None, -18000))]
  identifier: nwb_01
  session_start_time: 2019-10-20 00:00:00-05:00
  subject: subject <class 'pynwb.file.Subject'>
  timestamps_reference_time: 2019-10-20 00:00:00-05:00

## This concludes the basic showcase of using `dj.AttributeAdapter` to work with `NWB` objects

Continue further to see more examples, but the core usage is demonstrated above

In [17]:
# -- create NWB 
nwb2 = NWBFile(identifier='nwb_02',
              session_description='',
              session_start_time=datetime.strptime('2019-10-20', '%Y-%m-%d'),
              file_create_date=datetime.now(tzlocal()),
              experimenter='John Smith')
# -- add subject
nwb2.subject = pynwb.file.Subject(
    subject_id='animal_01',
    sex='F')

In [18]:
# -- create NWB 
nwb3 = NWBFile(identifier='nwb_03',
              session_description='',
              session_start_time=datetime.strptime('2019-10-20', '%Y-%m-%d'),
              file_create_date=datetime.now(tzlocal()),
              experimenter='John Smith')
# -- add subject
nwb3.subject = pynwb.file.Subject(
    subject_id='animal_01',
    sex='F')

In [19]:
NWB.insert([{'nwb_id': 2, 'nwb': nwb2},
            {'nwb_id': 3, 'nwb': nwb3}])

Write NWB 2.0 file: nwb_02.nwb
Write NWB 2.0 file: nwb_03.nwb


In [20]:
NWB()

nwb_id,nwb
3,=BLOB=
2,=BLOB=
0,=BLOB=


In [21]:
fetch_nwb3 = (NWB & 'nwb_id=3').fetch1('nwb')

In [22]:
fetch_nwb3


root <class 'pynwb.file.NWBFile'>
Fields:
  experimenter: ['John Smith']
  file_create_date: [datetime.datetime(2020, 5, 13, 16, 49, 10, 892192, tzinfo=tzoffset(None, -18000))]
  identifier: nwb_03
  session_start_time: 2019-10-20 00:00:00-05:00
  subject: subject <class 'pynwb.file.Subject'>
  timestamps_reference_time: 2019-10-20 00:00:00-05:00

### Let's also look at the directory where all the NWB files are generated (configured in the `nwbstore`)

In [24]:
os.listdir(exported_nwb_dir)

['nwb_01.nwb', 'nwb_02.nwb', 'nwb_03.nwb']